### Grabbing everything from email

- non-Python pre-req was assigning every update email to a specific label, and then using Google Takeout to download that folder in an .mbox format

In [94]:
import mailbox
import base64
import pandas as pd
import numpy as np
import re
import datetime as dt
from dateutil import tz 

In [95]:
def remove_email(string):
    index = [m.start() for m in re.finditer('<', string)]
    if len(index) > 0:
        return string[:index[0]-1]
    else: 
        return string

def decode_string(string):
    base64_bytes = string.encode("utf8") 
    sample_string_bytes = base64.b64decode(base64_bytes) 
    sample_string = sample_string_bytes.decode("utf8") 
    return sample_string

def subject_decode(subject_string):
    start_index = np.array([m.end() for m in re.finditer('=\?UTF-8\?B\?', subject_string)])
    if len(start_index) == 0:
        return subject_string
    else:
        end_index = np.array([m.end() for m in re.finditer('\?', subject_string)])
        decoded_string = ''
        for start in start_index:
            end = np.array([x for x in end_index if x > start]).min()
            decoded_string = decoded_string + decode_string(subject_string[start:end-1])
        return decoded_string
    
def getbody(message): #getting plain text 'email body'
    body = None
    if message.is_multipart():
        for part in message.walk():
            if part.is_multipart():
                for subpart in part.walk():
                    if subpart.get_content_type() == 'text/plain':
                        body = subpart.get_payload(decode=True)
            elif part.get_content_type() == 'text/plain':
                body = part.get_payload(decode=True)
    elif message.get_content_type() == 'text/plain':
        body = message.get_payload(decode=True)
    return body

In [177]:
mbox = mailbox.mbox("squid_updates.mbox")

df_email = pd.DataFrame(columns = ['author', 'date', 'subject', 'content'])

for i, message in enumerate(mbox):

    dict = {}
    dict['author'] = [message['from']]
    dict['date'] = [message['date']]
    dict['subject'] = [message['subject']]
    dict['content'] = [getbody(message)]

    df_email = pd.concat([df_email, pd.DataFrame.from_dict(dict)], axis = 0)

In [178]:
df_email_proc = (df_email
    # Fixing date format
    .assign(date = lambda x: pd.to_datetime(x['date'], format = '%a, %d %b %Y %H:%M:%S %z'))
    .sort_values(by='date')
    .assign(content = lambda x: x['content'].str.decode('latin1'))
    .assign(subject = lambda y: y.apply(lambda x: subject_decode(x['subject']), axis = 1))
    .assign(author = lambda y: y.apply(lambda x: remove_email(x['author']), axis = 1))
    .assign(author = lambda x: x['author'].replace({'Matthew Kendall':'Matt Kendall','zachredmond18@gmail.com':'Zach Hyman', 'bryanfiori2015@u.northwestern.edu':'Bryan Fiori Ryan'}))
    # Some basic filtering. Removing replies, short emails that shouldn't be in there, and the one time Sid forwarded my original update
    .loc[lambda x: ~x['subject'].str.startswith('Re:')]
    .loc[lambda x: x['subject']!='Fwd: Life Update Round 1']
)

zach_part_1_subject = df_email_proc.loc[(df_email_proc['author']=='Zach Hyman') & (df_email_proc['subject']=='Life Update 4, now with GIFs - Part 1') & (df_email_proc['date'] < dt.datetime(2016, 5, 10, tzinfo = tz.gettz('America/Chicago'))), 'subject'].values[0]
zach_part_2_subject = df_email_proc.loc[(df_email_proc['author']=='Zach Hyman') & (df_email_proc['subject']=='Life Update Part 2') & (df_email_proc['date'] < dt.datetime(2016, 5, 10, tzinfo = tz.gettz('America/Chicago'))), 'subject'].values[0]
zach_comb_subject = zach_part_1_subject + '\n' + zach_part_2_subject

zach_part_1_content = df_email_proc.loc[(df_email_proc['author']=='Zach Hyman') & (df_email_proc['subject']=='Life Update 4, now with GIFs - Part 1') & (df_email_proc['date'] < dt.datetime(2016, 5, 10, tzinfo = tz.gettz('America/Chicago'))), 'content'].values[0]
zach_part_2_content = df_email_proc.loc[(df_email_proc['author']=='Zach Hyman') & (df_email_proc['subject']=='Life Update Part 2') & (df_email_proc['date'] < dt.datetime(2016, 5, 10, tzinfo = tz.gettz('America/Chicago'))), 'content'].values[0]
zach_comb_content = zach_part_1_content + '\n' + zach_part_2_content

df_email_proc.loc[(df_email_proc['author']=='Zach Hyman') & (df_email_proc['subject']=='Life Update 4, now with GIFs - Part 1') & (df_email_proc['date'] < dt.datetime(2016, 5, 10, tzinfo = tz.gettz('America/Chicago'))), 'subject'] = zach_comb_subject
df_email_proc.loc[(df_email_proc['author']=='Zach Hyman') & (df_email_proc['subject']=='Life Update 4, now with GIFs - Part 1') & (df_email_proc['date'] < dt.datetime(2016, 5, 10, tzinfo = tz.gettz('America/Chicago'))), 'content'] = zach_comb_content

df_email_proc = df_email_proc.loc[~((df_email_proc['author']=='Zach Hyman') & (df_email_proc['subject']=='Life Update Part 2') & (df_email_proc['date'] < dt.datetime(2016, 5, 10, tzinfo = tz.gettz('America/Chicago')))), :]
df_email_proc = df_email_proc.reset_index(drop=True)

# df_email_proc.to_csv('squid_update_dwh.csv')

In [180]:
df_email_proc

,author,date,subject,content
0,Matthew Kendall,2015-10-05 13:13:36-05:00,Life Update Round 1,"Hey ya'll,\n\nHope your weeks are getting off ..."
1,Jordan Scott,2015-10-12 09:35:42-05:00,Life Update Round 2,HEY FUCKERS\n\n\nIâm going to mostly follow ...
2,Andrew Beir,2015-10-19 14:36:28-04:00,Life Update Round 3,"Hello fellow squids,\n\nHad a hell of a weeken..."
3,Daniel Ranti,2015-10-19 22:09:53-04:00,Life Update Part 3B,"Fellow Squids,\n\nBuckle up ricky fucking tick..."
4,Eric Yang,2015-10-26 11:46:49-05:00,Life Update Round 4,"Fellow Calamari,\n\nFirst off, I apologize for..."
...,...,...,...,...
293,Jordan Scott,2022-08-11 19:33:30-06:00,lets go brandon (tennis),"LGB(T) Qommunity,\n\nfor once, a lot has happe..."
294,Eric Yang,2022-08-18 22:40:24-07:00,Back to ɴᴏʀᴍᴀʟ Mode,Beep Boop. Doing something new this time aroun...
295,Daniel Ranti,2022-08-22 16:52:46-04:00,Are we still sending these via email too,https://docs.google.com/document/d/1jpnmv3u0TL...
296,"""Matt Felz (via Google Docs)""",2023-10-27 04:08:42+00:00,"Document shared with you: ""Felz 2023/10/26 Spo...",I've shared an item with you:\n\nFelz 2023/10/...


### Grabbing everything from Google docs

In [1]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [2]:
# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/documents.readonly", "https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file"]

In [3]:
# Grab credentials.json from here: https://console.cloud.google.com/apis/credentials?project=dbt-learn-370222
# Other documentation if needed: https://developers.google.com/docs/api/quickstart/python

creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists("token.json"):
  creds = Credentials.from_authorized_user_file("token.json", SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
  if creds and creds.expired and creds.refresh_token:
    creds.refresh(Request())
  else:
    flow = InstalledAppFlow.from_client_secrets_file(
        "credentials.json", SCOPES
    )
    creds = flow.run_local_server(port=0)
  # Save the credentials for the next run
  with open("token.json", "w") as token:
    token.write(creds.to_json())


FileNotFoundError: [Errno 2] No such file or directory: 'credentials.json'

In [239]:
def get_plain_text(DOCUMENT_ID, creds):

  try:
    service = build("docs", "v1", credentials=creds)

    # Retrieve the documents contents from the Docs service.
    document = service.documents().get(documentId=DOCUMENT_ID).execute()

    file = document.get('body')
    
    content_string = ''
    for element in file['content']:
        if 'paragraph' in element:
            paragraph = element['paragraph']
            for element in paragraph['elements']:
                if 'textRun' in element:
                    content_string = content_string + element['textRun']['content']

    return content_string

  except HttpError as err:
    print(err)

def get_file_info_from_id(fileId, service):
    try:

        response = service.files().get(fileId=fileId, fields="id, name, owners, createdTime").execute()

        df = pd.DataFrame(columns = ['author', 'date', 'subject', 'fileId'])

        item = response

        dict = {}
        dict['fileId'] = fileId
        dict['subject'] = item['name']
        dict['author'] = item['owners'][0]['displayName']

        created_at = pd.to_datetime(item['createdTime'])

        comment_dates = np.array([])
        comments = service.comments().list(fileId=fileId,fields='comments').execute()      
        for comment in comments.get('comments'):         
            comment_dates = np.append(comment_dates, pd.to_datetime(comment['createdTime']))

        if len(comment_dates) > 0:
            created_at = comment_dates.min()

        dict['date'] = created_at

        dict = {k:[v] for k,v in dict.items()}
        df = pd.concat([df, pd.DataFrame.from_dict(dict)], axis = 0)

        return df
    
    except HttpError as error:
        print(f"An error occurred: {error}")

def get_file_info_from_folder(folderId, creds):

  try:
    service = build("drive", "v3", credentials=creds)
  
    response = service.files().list(q=f"'{folderId}' in parents", pageSize=1000, fields="files(id, mimeType, name, owners, createdTime)").execute()
    items = response.get('files', [])

    df = pd.DataFrame(columns = ['author', 'date', 'subject', 'fileId'])

    for item in items:
      if item['mimeType'] == 'application/vnd.google-apps.document':
        dict = {}
        dict['fileId'] = item['id']
        dict['subject'] = item['name']
        dict['author'] = item['owners'][0]['displayName']

        created_at = pd.to_datetime(item['createdTime'])

        comment_dates = np.array([])
        comments = service.comments().list(fileId=item['id'],fields='comments').execute()      
        for comment in comments.get('comments'):         
          comment_dates = np.append(comment_dates, pd.to_datetime(comment['createdTime']))

        if len(comment_dates) > 0:
          created_at = comment_dates.min()

        dict['date'] = created_at

        dict = {k:[v] for k,v in dict.items()}
        df = pd.concat([df, pd.DataFrame.from_dict(dict)], axis = 0)

    return df
      
  except HttpError as error:
    print(f"An error occurred: {error}")

In [273]:
id_list = [
    '1hhpQ2wQ8jGLrfJnr237dNnhwvTLKib6Htfb2_qCQrns',
    '1jtEpNKolr3_bxhSWygp5BKcXAJr7ELY3ZQWIKy7aWec',
    '1ZvNQFsFPfPZh2rUq1fEI4D5FmZ-RJxXC5tbNvGNKQGo'
]

service = build("drive", "v3", credentials=creds)

df_drive_list = pd.DataFrame(columns = ['author', 'date', 'subject', 'fileId'])

for doc in id_list:
    df_drive_list = pd.concat([df_drive_list, get_file_info_from_id(doc, service)])

In [276]:
df_drive_folder = get_file_info_from_folder('1s2eD0iWge9Uur6GXRQjtHWgG47JryYkR', creds)

In [279]:
df_drive = pd.concat([df_drive_list, df_drive_folder]).drop_duplicates()

In [214]:
df_drive_proc = (
    df_drive
    .sort_values(by='date')
    .assign(content = lambda y: y.apply(lambda x: get_plain_text(x['fileId'], creds), axis = 1))
    .drop('fileId', axis=1)
)

df_drive_proc = df_drive_proc.loc[~((df_drive_proc['author']=='Jordan Scott') & (df_drive_proc['subject']=="[JORDAN] let's go brandon (tennis)") & (df_drive_proc['date'] < dt.datetime(2022, 8, 20, tzinfo = tz.gettz('America/Chicago')))), :]


In [215]:
df_comb = (
    pd.concat([df_email_proc, df_drive_proc])
    .assign(word_count = lambda x: x['content'].str.split().str.len())
    .loc[lambda x: x['word_count']>100]
    .sort_values(by='date')
    .assign(days_since_prior = lambda x: (x['date'] - x['date'].shift(1))/np.timedelta64(1, 'D'))
    .reset_index(drop=True)
)



In [222]:
df_comb.to_csv('squid_updates.csv', encoding = 'utf-8-sig')